# Unsupervised Classification of STM Images

In this notebook we will be training an autoencoder with MNIST images. This model will be saved and later used for future parts of the project.

---

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras import models

from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Reshape, UpSampling2D, Dropout, Conv2DTranspose, Activation, Concatenate
from keras.models import Model

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import math
import random
import cv2
import os

import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl

---

## Step 1: Importing Datasets & Re-shaping Images.

---

In [ ]:
### importing STM Images ###

# loading folder
path = "..."
image_folder = [f for f in os.listdir( ... ) if f.endswith((".jpg", ".png", ".jpeg"))]

# ----------- extracting images from folder ----------- #

all_images = []

for i, image_file in enumerate( image_folder ):
    
    image_path = os.path.join( path, image_file )
    img        = cv2.imread( image_path )
    
    all_images.append( img )

# normalising images
all_images = np.array( all_images )
all_images = all_images / np.max(all_images)


In [ ]:
### Defining Datasets ##

kTraining_images   = all_images[:600]    # 600
kValidation_images = all_images[600:-1]  # 150
kTesting_images    = all_images[-1:]     # 1


---

## Step 2: Creating and Training the Autoencoder.

- No optimisation techniques applied at first.

---

In [ ]:
### Autoencoder Model ###

# Needed for decoder #
shape_before_flattening = (4, 4, 256)

# -------------------------------------------------------------------------------- #

input_img_y = Input(shape=(64, 64, 3))

# Encoder #
y = Conv2D( 32, (3, 3), activation='relu', padding='same', strides=2)(input_img_y)
y = Conv2D( 64, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2D( 128, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2D( 256, (3, 3), activation='relu', padding='same', strides=2)(y)

# Latent layers #
y = Flatten()(y)
y = Dense( 32, activation='relu')(y)
y = Dense( np.prod(shape_before_flattening) )(y)
y = Reshape( shape_before_flattening )(y)

# Decoder #
y = Conv2DTranspose( 256, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 128, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 64, (3, 3), activation='relu', padding='same', strides=2)(y)
y = Conv2DTranspose( 32, (3, 3), activation='relu', padding='same', strides=2)(y)

output_y = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(y)


# ----------- Autoencoder model ----------- #
autoencoder_y = Model( input_img_y, output_y )
autoencoder_y.compile(optimizer="adam",
                      loss="mean_squared_error",
                      metrics=['mean_squared_error','accuracy'])

autoencoder_y.summary()


In [ ]:
### Training model ###

history = autoencoder_y.fit( kTraining_images, kTraining_images,
                            epochs=150,
                            validation_data=( kValidation_images, kValidation_images ));


In [ ]:
### PLOTTING ###

# mean squared error data
loss = history.history["mean_squared_error"]
val_loss = history.history["val_mean_squared_error"]

# x axis
epochs = range(1, len(loss) + 1)

# ---------------------------- Plotting ---------------------------- #

fig1 = plt.figure(figsize=(15,7.5))
ax2 = fig1.add_subplot(1,1,1)

ax2.plot(epochs, loss, "ro", label="Training")
ax2.plot(epochs, val_loss, "r", label="Validation")
ax2.legend()
ax2.grid()

ax2.set_ylabel('Mean squared error')
ax2.set_xlabel('Epochs')

y_labels = np.arange(0,0.016,0.001)  # Define your labels here
ax2.set_yticks(y_labels);



---

## Step 3: Testing and Saving model.

---

In [ ]:
### Testing Quality of Autoencoder Reconstruction ###

# choosing test images
constructed_image = kTesting_images[ 0 ]
constructed_image = np.expand_dims( constructed_image, axis=0 )

# applying autoencoder model
reconstructed_image = autoencoder_y.predict(constructed_image)

# -------- plotting --------- #
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,2,1)
ax2 = fig1.add_subplot(1,2,2)

ax1.imshow( constructed_image[0] , cmap='gray')
ax2.imshow( reconstructed_image[0] , cmap='gray');


In [ ]:
### ----------------------- SAVING MODEL ----------------------- ###

# autoencoder_y.save('autoencoder_model_stm')


---